In [1]:
## Libraries
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BertEmbedding
import numpy as np

In [2]:
## DATA
# SAVE_DATA_clus_tr = "Save/clus_bistlstm_dataset_train.csv"
# SAVE_DATA_clus_ts = "Save/clus_bistlstm_dataset_test.csv"
# SAVE_DATA_clus_dv = "Save/clus_bistlstm_dataset_dev.csv"

SAVE_DATA_noclus_tr = "Save/noclus_bistlstm_dataset_train.csv"
SAVE_DATA_noclus_ts = "Save/noclus_bistlstm_dataset_test.csv"
SAVE_DATA_noclus_dv = "Save/noclus_bistlstm_dataset_dev.csv"

train = open(SAVE_DATA_noclus_tr).read()
test = open(SAVE_DATA_noclus_ts).read()
val = open(SAVE_DATA_noclus_dv).read()

def file_convert2format(file):
    split_file = file.split("\n\t\n")
    t_x = []
    t_y = []
    for sent in split_file:
        t_x.append([])
        t_y.append([])
        sent_split = sent.split("\n")
        for lines in sent_split:
            line_split = lines.split("\t")
            t_x[-1].append(line_split[0])
            t_y[-1].append(line_split[1])
            
    return(t_x,t_y)

In [3]:
train_x, train_y = file_convert2format(train)
test_x, test_y = file_convert2format(test)
valid_x, valid_y = file_convert2format(val)

In [15]:
total_x = np.array(list(train_x + test_x + valid_x))
total_y = np.array(list(train_y + test_y + valid_y))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [16]:
print(len(total_x))
print(len(total_y))
index = np.array(list(range(len(total_x))))    

## shuffling the dataset
np.random.shuffle(index)
total_x = total_x[index]
total_y = total_y[index]

## dividing the dataset
train = int(len(total_x)*0.75)          ## 75% of the 100%
val = int((len(total_x) - train)*0.4)  ## 40% of the 30%
test = (len(total_x)-train-val)        ## 60% of the 30%
print(test,train,val)

## the dataset
## tokens
train_x = list(total_x[:train])
test_x = list(total_x[train:(train+test)])
valid_x = list(total_x[(train+test):])

## labels
train_y = list(total_y[:train])
test_y = list(total_y[train:(train+test)])
valid_y = list(total_y[(train+test):])


4108
4108
617 3081 410


In [12]:
## using pretrained embeddings to fine tune for task
## BERT_Tiny_2_128
## cased_L-12_H-768_A-12

bert_embed = BertEmbedding('BERTmodels/BERT_Tiny_2_128')
model = BiLSTM_Model()
hyper = model.default_hyper_parameters()

hyper['layer_blstm']['units'] = 64
hyper['layer_dropout']['rate'] = 0.35
hyper['layer_activation']['activation'] = 'softmax'
model = BiLSTM_Model(bert_embed, sequence_length=64,hyper_parameters=hyper)

print(hyper)

2021-05-05 04:26:35,036 [DEBUG] kashgari - ------------------------------------------------
2021-05-05 04:26:35,037 [DEBUG] kashgari - Loaded transformer model's vocab
2021-05-05 04:26:35,038 [DEBUG] kashgari - config_path       : BERTmodels/BERT_Tiny_2_128/bert_config.json
2021-05-05 04:26:35,039 [DEBUG] kashgari - vocab_path      : BERTmodels/BERT_Tiny_2_128/vocab.txt
2021-05-05 04:26:35,040 [DEBUG] kashgari - checkpoint_path : BERTmodels/BERT_Tiny_2_128/bert_model.ckpt
2021-05-05 04:26:35,040 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', 

{'layer_blstm': {'units': 64, 'return_sequences': True}, 'layer_dropout': {'rate': 0.35}, 'layer_time_distributed': {}, 'layer_activation': {'activation': 'softmax'}}


In [13]:
model.fit(train_x, train_y, valid_x, valid_y)

Preparing text vocab dict: 100%|██████████| 410/410 [00:00<00:00, 89720.07it/s]
2021-05-05 04:26:35,786 [DEBUG] kashgari - --- Build vocab dict finished, Total: 3319 ---
2021-05-05 04:26:35,787 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', 'the', ',', '.', 'to', 'of', 'and']
Preparing text vocab dict: 100%|██████████| 410/410 [00:00<00:00, 112337.64it/s]
2021-05-05 04:26:35,823 [DEBUG] kashgari - --- Build vocab dict finished, Total: 7 ---
2021-05-05 04:26:35,823 [DEBUG] kashgari - Top-10: ['[PAD]', 'O', 'I-CLAIM', 'I-ACT', 'B-CLAIM', 'B-ACT', '']
2021-05-05 04:26:38,064 [DEBUG] kashgari - fit input shape: (2, 64, 64)
2021-05-05 04:26:38,066 [DEBUG] kashgari - fit input shape: (64, 64)


Epoch 1/5
48/48 [==============================] - 15s 147ms/step - loss: 0.3379 - accuracy: 0.7817 - val_loss: 0.1718 - val_accuracy: 0.9048
Epoch 2/5
48/48 [==============================] - 4s 89ms/step - loss: 0.1541 - accuracy: 0.9155 - val_loss: 0.1321 - val_accuracy: 0.9095
Epoch 3/5
48/48 [==============================] - 4s 89ms/step - loss: 0.1307 - accuracy: 0.9218 - val_loss: 0.1145 - val_accuracy: 0.9190
Epoch 4/5
48/48 [==============================] - 4s 90ms/step - loss: 0.1291 - accuracy: 0.9207 - val_loss: 0.1210 - val_accuracy: 0.9130
Epoch 5/5
48/48 [==============================] - 4s 90ms/step - loss: 0.1189 - accuracy: 0.9210 - val_loss: 0.1179 - val_accuracy: 0.9108


In [14]:
# Evaluate the model
model.evaluate(test_x, test_y)

2021-05-05 04:27:10,634 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 88
2021-05-05 04:27:10,650 [DEBUG] kashgari - predict seq_length: None, input: (2, 617, 88)


20/20 [==============================] - 3s 41ms/step


2021-05-05 04:27:14,165 [DEBUG] kashgari - predict output: (617, 88)
2021-05-05 04:27:14,166 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]]



           precision    recall  f1-score   support

      ACT     0.0000    0.0000    0.0000       102
    CLAIM     0.0690    0.0132    0.0222       151

micro avg     0.0690    0.0079    0.0142       253
macro avg     0.0412    0.0079    0.0133       253



{'detail': {'ACT': {'precision': 0,
   'recall': 0.0,
   'f1-score': 0,
   'support': 102},
  'CLAIM': {'precision': 0.06896551724137931,
   'recall': 0.013245033112582781,
   'f1-score': 0.022222222222222223,
   'support': 151}},
 'precision': 0.04116123756303666,
 'recall': 0.007905138339920948,
 'f1-score': 0.01326306543697848,
 'support': 253}

In [17]:
# Model data will save to 'saved_act_claim_model' folder
model.save('saved_act_claim_model_relu_1')
loaded_model = BiLSTM_Model.load_model('saved_act_claim_model_relu_1')

2021-05-05 04:51:07,829 [INFO] kashgari - model saved to /project/ClimaChange_Claims/saved_act_claim_model_relu_1
2021-05-05 04:51:07,936 [DEBUG] kashgari - ------------------------------------------------
2021-05-05 04:51:07,937 [DEBUG] kashgari - Loaded transformer model's vocab
2021-05-05 04:51:07,938 [DEBUG] kashgari - config_path       : BERTmodels/BERT_Tiny_2_128/bert_config.json
2021-05-05 04:51:07,939 [DEBUG] kashgari - vocab_path      : BERTmodels/BERT_Tiny_2_128/vocab.txt
2021-05-05 04:51:07,940 [DEBUG] kashgari - checkpoint_path : BERTmodels/BERT_Tiny_2_128/bert_model.ckpt
2021-05-05 04:51:07,940 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]'

In [20]:
# Load saved model
print(test_x[12])
loaded_model.predict(test_x[11:12])

2021-05-05 04:52:09,322 [DEBUG] kashgari - predict seq_length: None, input: (2, 1, 19)


['Thinking', 'further', ',', 'he', 'adds', ':', "'", "'", 'The', 'problem', 'is', ',', 'we', "'ve", 'gone', 'too', 'far', 'already', '.']
1/1 [==============================] - 3s 3s/step


2021-05-05 04:52:11,931 [DEBUG] kashgari - predict output: (1, 19)
2021-05-05 04:52:11,933 [DEBUG] kashgari - predict output argmax: [[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]]


[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]